# Initialization and Config


In [4]:
%load_ext autoreload
%autoreload 2

import torch
import gc
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import logging
import wandb
import os
import sys
from dotenv import load_dotenv
from typing import Literal

sys.path.append("..")
from Datasets.coco import CocoDataset
from Datasets.cc12m import CC12mDataset
from Datasets.cood import CoodDataset
from Datasets.laion import LaionDataset
from Models.clipModel import CLIPModel
from Models.cloobModel import CLOOBModel
from Models.vClipModel import VariationalCLIPModel
from Models.alignClipModel import AlignCLIPModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'cloob_training'

In [ ]:
type Model = Literal['CLIP', 'CLOOB', 'ALIGN', 'VCLIP']
type ModelClass = CLIPModel | CLOOBModel | AlignCLIPModel | VariationalCLIPModel
MODEL: Model = 'CLIP'

type Dataset = Literal['COCO', 'COOD', 'CC12M', 'LAION']
DATASET: Dataset = 'COCO'

DEVICE = None
if torch.cuda.is_available():
    DEVICE = 'cuda'
elif torch.mps.is_available():
    DEVICE = 'mps'
else:
    DEVICE = 'cpu'

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Get WANDB Key
load_dotenv()
WANDB_API_KEY = os.environ.get('WANDB_API_KEY')

In [ ]:
CONFIG = {
    'BATCH_SIZE': 50,
    'LEARNING_RATE': 1e-3,
    'WEIGHT_DECAY': 1e-1,

    'STEP_LR_STEP_SIZE': 5,
    'STEP_LR_GAMMA': 0.5,

    # To avoid gradient explosion. Set to 1 to disable
    'GRAD_ACCUMULATION_STEPS': 1,
    'CLIP_GRADIENTS': False,
    'EMPTY_CACHE_AFTER_BATCH': False,

    'TRAIN_RATIO': 0.8,
    'DATA_DIR': '../Data',
    'TOTAL_DATAPOINTS': 10_000,

    'USE_LORA': False,

    'USE_WANDB': True,
    'WANDB_RUN_NAME': 'experiment_1',
    'WANDB_PREVIOUS_RUN_ID': None, # set to None if not resuming
    'WANDB_PROJECT_NAME': 'multimodal_2025',
}

# Data


In [ ]:
num_train = int(CONFIG['TOTAL_DATAPOINTS'] * CONFIG['TRAIN_RATIO'])
num_val = int(CONFIG['TOTAL_DATAPOINTS'] * (1.0 - CONFIG['TRAIN_RATIO']))

if DATASET == 'COCO':
    train = CocoDataset(
        data_dir=CONFIG['DATA_DIR'],
        split='train2017',
        tokenize=True,
        max_samples=num_train
    )
    val = CocoDataset(
        data_dir=CONFIG['DATA_DIR'],
        split='val2017',
        tokenize=True,
        max_samples=num_val
    )   
elif DATASET == 'COOD':
    all_data = CoodDataset(
        data_dir=CONFIG['DATA_DIR'],
        tokenize=True,
        max_samples=CONFIG['TOTAL_DATAPOINTS']
    )
    train = torch.utils.data.Subset(
        all_data,
        range(0, num_train)
    )
    val = torch.utils.data.Subset(
        all_data,
        range(num_train, CONFIG['TOTAL_DATAPOINTS'])
    )
elif DATASET == 'LAION':
    all_data = LaionDataset(
        tokenize=True,
        max_samples=CONFIG['TOTAL_DATAPOINTS']
    )
    train = torch.utils.data.Subset(
        all_data,
        range(0, num_train)
    )
    val = torch.utils.data.Subset(
        all_data,
        range(num_train, CONFIG['TOTAL_DATAPOINTS'])
    )

elif DATASET == 'CC12M':
    all_data = CC12mDataset(
        data_dir=CONFIG['DATA_DIR'],
        tokenize=True,
        max_samples=CONFIG['TOTAL_DATAPOINTS']
    )
    train = torch.utils.data.Subset(
        all_data,
        range(0, num_train)
    )
    val = torch.utils.data.Subset(
        all_data,
        range(num_train, CONFIG['TOTAL_DATAPOINTS'])
    )
    


In [ ]:
train_loader = torch.utils.data.DataLoader(
    train,
    CONFIG['BATCH_SIZE'],
    shuffle=True,
    num_workers=0,
    pin_memory=DEVICE == 'cuda'
)
val_loader = torch.utils.data.DataLoader(
    val,
    CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=0,
    pin_memory=DEVICE == 'cuda'
)

# Model


In [ ]:
from losses.clipLoss import ClipLoss
from losses.cloobLoss import CLOOBLoss

if MODEL == 'CLIP':
    model = CLIPModel(
        device=DEVICE,
    )
    loss = ClipLoss()
elif MODEL == 'CLOOB':
    model = CLOOBModel(
        device=DEVICE
    )
    config = model.get_config()
    loss = CLOOBLoss(config['inv_tau'], config['scale_hopfield'])
elif MODEL == 'VCLIP':
    model = VariationalCLIPModel(
        device=DEVICE
    )
    loss = None
else:
    model = AlignCLIPModel(
        device=DEVICE
    )
    loss = None
    

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CONFIG['LEARNING_RATE'],
    weight_decay=CONFIG['WEIGHT_DECAY']
)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=CONFIG['STEP_LR_STEP_SIZE'],
    gamma=CONFIG['STEP_LR_GAMMA']
)

# Training


In [ ]:
def train_epoch(
        model: ModelClass, 
        dataloader: torch.utils.data.DataLoader, 
        optimizer: torch.optim.Optimizer, 
        criterion: torch.nn.Module, 
):
    model.train()

    progress_bar = tqdm.tqdm(dataloader, desc="Training Epoch")
    total_loss = 0.0
    nan_count = 0

    for batch_idx, (images, text_tokens) in enumerate(progress_bar):
        images, text_tokens = images.to(DEVICE), text_tokens.to(DEVICE)
    
        # Check for NaN in input (laion gives NAN's if it. can't load images)
        if torch.isnan(images).any() or torch.isnan(text_tokens).any():
            logger.warning(f"NaN in input batch {batch_idx}")
            optimizer.zero_grad()
            continue
        
        image_features, text_features = model(images)
    
        # Check for NaN in features
        if torch.isnan(image_features).any() or torch.isnan(text_features).any():
            logger.warning(f"NaN in features at batch {batch_idx}")
            nan_count += 1
            optimizer.zero_grad()
            continue
        
        loss = criterion(image_features, text_features)

        # Check for NaN in loss
        if torch.isnan(loss):
            logger.warning(f"NaN loss detected at batch {batch_idx}")
            nan_count += 1
            optimizer.zero_grad()
            continue
        
        # If not NaN, then add to total loss
        total_loss += loss.item()
    
        # Scale loss for gradient accumulation
        scaled_loss = loss / CONFIG['GRAD_ACCUMULATION_STEPS']
        # Backward pass
        scaled_loss.backward()

        has_nan_grads = False
        for name, param in model.named_parameters():
            if param.grad is not None and torch.isnan(param.grad).any():
                has_nan_grads = True
                logger.warning(f"NaN gradient in {name}")
                break

        if has_nan_grads:
            logger.warning(f"NaN gradients detected at batch {batch_idx}, skipping update")
            optimizer.zero_grad()
            continue

        # Gradient accumulation and optimization step
        if (batch_idx + 1) % CONFIG['GRAD_ACCUMULATION_STEPS'] == 0:
            # Clip gradients to prevent explosion
            if CONFIG['CLIP_GRADIENTS']:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            optimizer.zero_grad()

            progress_bar.set_postfix({
                'loss': total_loss / (batch_idx + 1),
                  'nan_count': nan_count
            })


        if CONFIG['EMPTY_CACHE_AFTER_BATCH']:
            torch.cuda.empty_cache()
            gc.collect()

    # Handle remaining gradients if not accumulated evenly
    if len(dataloader) % CONFIG['GRAD_ACCUMULATION_STEPS'] != 0:
        if CONFIG['CLIP_GRADIENTS']:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        optimizer.zero_grad()

    epoch_loss = total_loss / len(dataloader)
    return epoch_loss

# Validation

In [ ]:
def validate(
        model: ModelClass, 
        dataloader: torch.utils.data.DataLoader, 
        criterion: torch.nn.Module,
):
    model.eval()

    progress_bar = tqdm.tqdm(dataloader, desc="Evaluating")
    total_loss = 0.0

    with torch.no_grad():
        for batch_idx, (images, text_tokens) in enumerate(progress_bar):
            images, text_tokens = images.to(DEVICE), text_tokens.to(DEVICE)
    
            # Check for NaN in input (laion gives NAN's if it. can't load images)
            if torch.isnan(images).any() or torch.isnan(text_tokens).any():
                logger.warning(f"NaN in input batch {batch_idx}")
                optimizer.zero_grad()
                continue
        
            image_features, text_features = model(images)

            # Check for NaN in features
            if torch.isnan(image_features).any() or torch.isnan(text_features).any():
                logger.warning(f"NaN in features at batch {batch_idx}")
                optimizer.zero_grad()
                continue
        
            loss = criterion(image_features, text_features)

            # Check for NaN loss
            if torch.isnan(loss):
                logger.warning("NaN in validation loss, skipping batch")
                continue
        
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': total_loss / (batch_idx + 1)})

            if CONFIG['EMPTY_CACHE_AFTER_BATCH']:
                torch.cuda.empty_cache()
                gc.collect()

    epoch_loss = total_loss / len(dataloader)
    return epoch_loss


# Full Train Eval Pipeline

In [ ]:
def save_checkpoint(
    filename: str,
    model: ModelClass, 
    optimizer: torch.optim.Optimizer, 
    train_losses: list[float],
    val_losses: list[float],
):
    """Save model checkpoint."""
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses,
        'val_losses': val_losses,
        'config': CONFIG,
    }
    torch.save(checkpoint, filename)
    logger.info(f"Checkpoint saved: {filename}")

def plot_losses(model_name: str, train_losses: list[float], val_losses: list[float]):
    """Plot training and validation losses."""
    if len(train_losses) == 0:
        logger.warning("No losses to plot")
        return

    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss', marker='o')
    plt.plot(val_losses, label='Val Loss', marker='s')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{model_name.upper()} Training Curves')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def train( 
    num_epochs: int,
    model: ModelClass, 
    train_loader: torch.utils.data.DataLoader,
    val_loader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer, 
    scheduler: torch.optim.lr_scheduler.LRScheduler,
    criterion: torch.nn.Module
):
    """Train model for specified epochs."""
    logger.info(f"Starting training on {DEVICE} for {num_epochs} epochs...")

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        logger.info(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Train
        train_loss = train_epoch(
            model,
            train_loader,
            optimizer,
    criterion
        )
        train_losses.append(train_loss)
        logger.info(f"Train Loss: {train_loss:.6f}")

        # Validate
        val_loss = validate(
            model,
            val_loader,
            criterion
        )
        val_losses.append(val_loss)
        logger.info(f"Val Loss: {val_loss:.6f}")

        # Skip if losses are NaN
        if torch.isnan(torch.tensor(train_loss)) or torch.isnan(torch.tensor(val_loss)):
            logger.error("NaN loss detected! Stopping training.")
            break

        scheduler.step()
        logger.info(f"Learning Rate adjusted to: {scheduler.get_last_lr()[0]:.6f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_checkpoint(
                f"best_{MODEL}_model.pt", 
                model,
                optimizer, 
                train_losses,
                val_losses
            )
            logger.info(f"✓ Saved best model (val_loss: {val_loss:.6f})")

        if CONFIG['USE_WANDB']:
            wandb.log({
                'epoch': epoch + 1,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'learning_rate': scheduler.get_last_lr()[0],
            })

    return train_losses, val_losses

# WandB

In [ ]:
# Use wandb? Resume Training?
PROJECT_NAME = CONFIG['WANDB_PROJECT_NAME']
USE_WANDB = CONFIG['USE_WANDB']
RESUME_LOGGING = CONFIG['WANDB_PREVIOUS_RUN_ID'] is not None
run_name = CONFIG['WANDB_RUN_NAME']

if USE_WANDB:
    wandb.login(key=WANDB_API_KEY)  # your wandb key

    if RESUME_LOGGING:
        run_id = CONFIG['WANDB_PREVIOUS_RUN_ID']
        run = wandb.init(
            settings=wandb.Settings(symlink=False),
            id=run_id,
            resume="must",
            project=PROJECT_NAME,
        )
    else:
        run = wandb.init(
            name=run_name,
            reinit=True,
            project=PROJECT_NAME,
            config=CONFIG
        )

# Run Training


In [ ]:
train_losses, val_losses = train(
    CONFIG["NUM_EPOCHS"],
    model,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    loss # type: ignore
)

In [ ]:
plot_losses(MODEL, train_losses, val_losses)